In [1]:
!pip install wandb --upgrade

Requirement already up-to-date: wandb in /usr/local/lib/python3.7/dist-packages (0.10.33)


In [2]:
import numpy as np
import time
import matplotlib.pyplot as plt
import torch
from tqdm.notebook import tqdm_notebook
import torch.nn.functional as F
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader


if torch.cuda.is_available():
  device = torch.device("cuda:0")
  torch.backends.cudnn.deterministic = True
else:
  device = torch.device("cpu")

In [3]:
import wandb
wandb.login()

wandb: Currently logged in as: saiamrit (use `wandb login --relogin` to force relogin)


True

In [64]:
config = dict(
    epochs= 30,
    input = 784,
    hidden1 = 512,
    hidden2 = 512,
    output = 10,
    batch_size = 128,
    learning_rate = 0.9,
    dataset="MNIST",
    architecture="Logistic Regression"
)

In [65]:
def model_pipeline(hyperparameters):
  # start wandb experiment tracking
  with wandb.init(project = 'logistic-regression on mnist', config = hyperparameters):
    # to ensure logging Hyper parameters match execution
    config = wandb.config

    # initialise dataloaders, model, loss function and optimizer
    train_dataloader, test_dataloader, model, criteria, optimizer = create(config)
    print(model)

    # train and test the model
    train(train_dataloader, test_dataloader, model, criteria, optimizer, config)

    # save the model
    # torch.onnx.export(model, images, "model.onnx")
    wandb.save("model.onnx")

In [66]:
def create(config):
  train = datasets.MNIST(root = '/data', download = True, transform = transforms.ToTensor(), train = True)
  test = datasets.MNIST(root = '/data', transform = transforms.ToTensor(), train = False)

  train_dataloader = DataLoader(dataset = train, batch_size = config.batch_size, shuffle = True)
  test_dataloader = DataLoader(dataset = test, batch_size = config.batch_size, shuffle = False)
  
  torch.manual_seed(42)
  model = LogisticRegressionModel(config.input, config.hidden1, config.hidden2, config.output).to(device)

  criteria = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr = config.learning_rate)

  return train_dataloader, test_dataloader, model, criteria, optimizer

In [67]:
# # Hyperparameters
# random_seed = 1
# batch_size = 64
# learning_rate = 0.01
# epochs = 20

#Architecture
# input = 784
# hidden1 = 512
# hidden2 = 256
# output = 10

In [68]:
# train = datasets.MNIST(root = '/data', download = True, transform = transforms.ToTensor(), train = True)
# test = datasets.MNIST(root = '/data', transform = transforms.ToTensor(), train = False)

# train_dataloader = DataLoader(dataset = train, batch_size = batch_size, shuffle = True)
# test_dataloader = DataLoader(dataset = test, batch_size = batch_size, shuffle = False)

In [69]:
class LogisticRegressionModel(nn.Module):
  def __init__(self, input, hidden1, hidden2, output):
    super(LogisticRegressionModel, self).__init__()

    self.lin1 = nn.Linear(input, hidden1)
    self.lin2 = nn.Linear(hidden1, hidden2)
    self.lin3 = nn.Linear(hidden2, output)

  def forward(self, x):
    x = F.relu(self.lin1(x))
    x = F.relu(self.lin2(x))
    out = self.lin3(x)

    return out

# torch.manual_seed(random_seed)
# model = LogisticRegressionModel(input, output).to(device)

In [70]:
# criteria = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [71]:
def compute_accuracy(model, dataloader):
  correct = 0
  total = 0

  model.eval()
  with torch.no_grad():
    for images, labels in dataloader:
      images = images.view(-1, 28*28).to(device)
      labels = labels.to(device)
      
      outputs = model(images)

      preds = F.softmax(outputs)
      _, predictions = torch.max(preds,1)

      total += len(labels)
      correct += (predictions == labels).sum()

      return (correct/total)*100

In [75]:
def train(train_dataloader, test_dataloader, model, criterion, optimizer, config):
  model.train()
  wandb.watch(model, criterion, log = "all", log_freq = 10)
  start = time.time()
  # ep = []
  # train_loss = []
  # train_acc = []
  for epoch in tqdm_notebook(range(config.epochs), desc = 'Epoch progress'):
    for batch_idx, (images, labels) in tqdm_notebook(enumerate(train_dataloader), desc = 'Batch Training'):
      images = images.view(-1, 28*28).to(device)
      labels = labels.to(device)

      outputs = model(images)

      loss = criterion(outputs, labels)
      # ep.append(epoch)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if batch_idx % 300 == 0:
        print('Epoch: {}/{} | Batch: {}/{} | Train Loss = {:.4f}'.format(epoch, config.epochs, batch_idx, len(train_dataloader), loss))
    acc = compute_accuracy(model, train_dataloader)  
    print('For Epoch: {}/{}, Training Accuracy is: {}'.format(epoch, config.epochs, acc))
    wandb.log({"training accuracy": acc, "training loss": loss}, step = epoch)
    test(model, test_dataloader)
    # ep.append(epoch); train_acc.append(acc); train_loss.append(loss)
    print('Time taken for epoch {}/{} : {}'.format(epoch, config.epochs, (time.time()-start)/60))
  print('Total Training Time taken : {}'.format((time.time()-start)/60))
  # wandb.save("model.pt")

In [76]:
def test(model, test_dataloader):
  model.eval()

  with torch.no_grad():
    test_acc = compute_accuracy(model, test_dataloader)

    print("Test Accuracy is: ", test_acc)
    wandb.log({"test_accuracy": test_acc})

In [77]:
# plt.plot(ep,train_loss)
# plt.plot(ep,train_acc)
# plt.plot()

In [78]:
model = model_pipeline(config)

LogisticRegressionModel(
  (lin1): Linear(in_features=784, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=512, bias=True)
  (lin3): Linear(in_features=512, out_features=10, bias=True)
)


Epoch: 0/30 | Batch: 0/469 | Train Loss = 2.3007
Epoch: 0/30 | Batch: 300/469 | Train Loss = 0.1158

For Epoch: 0/30, Training Accuracy is: 96.875
Test Accuracy is:  tensor(96.0938, device='cuda:0')
Time taken for epoch 0/30 : 0.09850426912307739


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


Epoch: 1/30 | Batch: 0/469 | Train Loss = 0.1529
Epoch: 1/30 | Batch: 300/469 | Train Loss = 0.1097

For Epoch: 1/30, Training Accuracy is: 100.0
Test Accuracy is:  tensor(100., device='cuda:0')
Time taken for epoch 1/30 : 0.19310096899668375


Epoch: 2/30 | Batch: 0/469 | Train Loss = 0.0310
Epoch: 2/30 | Batch: 300/469 | Train Loss = 0.0500

For Epoch: 2/30, Training Accuracy is: 96.875
Test Accuracy is:  tensor(96.8750, device='cuda:0')
Time taken for epoch 2/30 : 0.28942071199417113


...

Epoch: 27/30 | Batch: 0/469 | Train Loss = 0.0001
Epoch: 27/30 | Batch: 300/469 | Train Loss = 0.0004

For Epoch: 27/30, Training Accuracy is: 100.0
Test Accuracy is:  tensor(100., device='cuda:0')
Time taken for epoch 27/30 : 2.7041593551635743


Epoch: 28/30 | Batch: 0/469 | Train Loss = 0.0001
Epoch: 28/30 | Batch: 300/469 | Train Loss = 0.0001

For Epoch: 28/30, Training Accuracy is: 100.0
Test Accuracy is:  tensor(100., device='cuda:0')
Time taken for epoch 28/30 : 2.8019798080126446


Epoch: 29/30 | Batch: 0/469 | Train Loss = 0.0001
Epoch: 29/30 | Batch: 300/469 | Train Loss = 0.0000

For Epoch: 29/30, Training Accuracy is: 100.0
Test Accuracy is:  tensor(100., device='cuda:0')
Time taken for epoch 29/30 : 2.898721480369568

Total Training Time taken : 2.898796832561493


training accuracy,100.0
training loss,9e-05
test_accuracy,100.0
_runtime,177
_timestamp,1625336398
_step,29


training accuracy,▁█▁▅█▅▅▆▆▆█▆██████████████████
training loss,█▆▅▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁█▂█▇██▇▇█▇▇██████████████████
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇██
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇██
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
